# 1

类似功能开发，如果直接给出所有数字，排序就可以依次扫描分区。

不确定的添加需要一个合理的数据结构，减少查询和插入的开销。

使用字典和列表记录每个值的区间。

![](../imgs/352.svg)

✘

In [ ]:
#include <iostream>
#include <vector>
#include <map>
using namespace std;

class SummaryRanges {
public:
    
    map<int, vector<int>> interval_dict;
    vector<vector<int>> ans;

    SummaryRanges() {
    }
    
    void addNum(int val) {
        map<int, vector<int>>::iterator it_pre = interval_dict.find(val-1);
        map<int, vector<int>>::iterator it_back = interval_dict.find(val+1);
        if(interval_dict.find(val) != interval_dict.end()){                     // 1.要添加的元素存在
            return; // 存在，不重复添加
        }
        
        if(it_pre != interval_dict.end() || it_back != interval_dict.end()){    // 2.要添加的元素前后至少存在一个情况
            vector<int> *tmp = nullptr;
            if(it_pre != interval_dict.end()){      // val-1元素存在
                tmp = &interval_dict[val-1];
                (*tmp)[1] = val;
            }
            if(it_back != interval_dict.end()){     // val+1元素存在
                if(tmp != nullptr){
                    (*tmp)[1] = interval_dict[val+1][1];
                    // interval_dict[val+1] = (*tmp);  // 后一个指向前一个  // 应该取地址
                    auto p_back = &interval_dict[val+1];
                    p_back = tmp;
                }else{
                    tmp = &interval_dict[val+1];
                    (*tmp)[0] = val;
                }
                // interval_dict.erase(val+1);     // 保留前一个，删除后一个    // 因为后续一个区间的元素都会指向同一个，如果这样移除可能会导致被清除，因为使用的标准库，所以应该会自动清除没引用的元素，应该不会内存泄露
            }
            interval_dict[val] = (*tmp); // 应该取地址
            // auto p_cur = &interval_dict[val];   // error，键为val不存在
            // p_cur = tmp;
        }else{                                                                  // 3.要添加的元素，以及前后都不存在
            interval_dict[val] = {val, val};
            ans.push_back(interval_dict[val]);
        }
    }
    
    vector<vector<int>> getIntervals() {
        return ans;
    }
};

int main(int argc, char const *argv[])
{
    SummaryRanges* obj = new SummaryRanges();
    obj->addNum(1);
    obj->getIntervals();
    obj->addNum(3);
    obj->getIntervals();
    obj->addNum(7);
    obj->getIntervals();
    obj->addNum(2);
    obj->getIntervals();
    obj->addNum(6);
    obj->getIntervals();
    obj->addNum(9);
    obj->getIntervals();
    obj->addNum(10);
    obj->getIntervals();
    obj->addNum(8);
    obj->getIntervals();

    return 0;
}

代码写的有问题，没有能共享，\
map<int, vector<int>>依旧是值传递，\
map<int, vector<int>*>应该使用指针，实现引用。

In [ ]:
#include <iostream>
#include <vector>
#include <map>
using namespace std;


class SummaryRanges {
public:
    
    map<int, vector<int>*> interval_dict;

    SummaryRanges() {
    }
    
    void addNum(int val) {
        map<int, vector<int>*>::iterator it_pre = interval_dict.find(val-1);
        map<int, vector<int>*>::iterator it_back = interval_dict.find(val+1);
        if(interval_dict.find(val) != interval_dict.end()){                     // 1.要添加的元素存在
            return; // 存在，不重复添加
        }

        if(it_pre != interval_dict.end() || it_back != interval_dict.end()){    // 2.要添加的元素前后至少存在一个情况
            vector<int> *tmp = nullptr;
            if(it_pre != interval_dict.end()){      // val-1元素存在    // 连接前面部分
                tmp = interval_dict[val-1];
                (*tmp)[1] = val;
            }
            if(it_back != interval_dict.end()){     // val+1元素存在
                if(tmp != nullptr){                 // 连接前后
                    (*tmp)[1] = (*interval_dict[val+1])[1];
                    auto p_back = interval_dict[val+1];
                    delete p_back;                  // 释放掉后面部分申请的空间
                    for(int i=val+1; i <= (*tmp)[1]; ++i){ // 往后遍历，后面属于区间内的重新指向
                        interval_dict[i] = tmp;
                    }
                }else{                              // 连接后面部分
                    tmp = interval_dict[val+1];
                    (*tmp)[0] = val;
                }
            }
            interval_dict[val] = tmp;
        }else{                                                                  // 3.要添加的元素和前后元素都不存在，分配空间添加
            interval_dict[val] = new vector<int>{val, val};
        }
    }
    
    vector<vector<int>> getIntervals() {
        vector<vector<int>> ans;
        int pre = -1;
        for(auto d : interval_dict){
            if((*d.second)[1] != pre){
                ans.push_back(*d.second);
                pre = (*d.second)[1];
            }
        }
        return ans;
    }
};


int main(int argc, char const *argv[])
{
    SummaryRanges* obj = new SummaryRanges();
    obj->addNum(1);
    obj->getIntervals();
    obj->addNum(3);
    obj->getIntervals();
    obj->addNum(7);
    obj->getIntervals();
    obj->addNum(2);
    obj->getIntervals();
    obj->addNum(6);
    obj->getIntervals();
    obj->addNum(9);
    obj->getIntervals();
    obj->addNum(10);
    obj->getIntervals();
    obj->addNum(5);
    obj->getIntervals();
    obj->addNum(11);
    obj->getIntervals();
    obj->addNum(8);
    obj->getIntervals();

    for(auto d : obj->interval_dict){
        cout << d.first << " " << (*d.second)[0] << " " << (*d.second)[1] << endl;
    }

    return 0;
}

9/9 cases passed (76 ms)\
Your runtime beats 24.67 % of cpp submissions\
Your memory usage beats 23.89 % of cpp submissions (32.8 MB)

addNum() - time: O(n)\
getIntervals() - time: O(n)

space: O(n)

做复杂的数据结构，插入看似近乎O(1)，考虑到向后的遍历，没有优点。

为了简单，在getIntervals中也没有从中获益。如果能在addNum中将结果同时操作，哪么应该会有些提升。

# 题解1

分五种情况：\
情况一： l_0 <= val <= l_1 \
情况二： r_0 + 1 = val \
情况三： l_1 - 1 = val \
情况四： r_0 + 1 = val 并且 l_1 - 1 = val \
情况五： 上述情况均不满足。

题解采用`map<int, int>`作为其数据结构，\
第一个int作为前区间，第二个int作为后区间，\
使用`upper_bound`二分查找map，时间复杂度为O(logn)。

然后根据不同情况进行处理。

> 相比自己使用的数据结构，更加便捷直观高效。（map居然有标准库二分查找）

In [ ]:
#include<map>
#include<vector>

using namespace std;

class SummaryRanges {
private:
    map<int, int> intervals;

public:
    SummaryRanges() {}
    
    void addNum(int val) {
        // 找到 l1 最小的且满足 l1 > val 的区间 interval1 = [l1, r1]
        // 如果不存在这样的区间，interval1 为尾迭代器
        auto interval1 = intervals.upper_bound(val);
        // 找到 l0 最大的且满足 l0 <= val 的区间 interval0 = [l0, r0]
        // 在有序集合中，interval0 就是 interval1 的前一个区间
        // 如果不存在这样的区间，interval0 为尾迭代器
        auto interval0 = (interval1 == intervals.begin() ? intervals.end() : prev(interval1));

        if (interval0 != intervals.end() && interval0->first <= val && val <= interval0->second) {
            // 情况一
            return;
        }
        else {
            bool left_aside = (interval0 != intervals.end() && interval0->second + 1 == val);
            bool right_aside = (interval1 != intervals.end() && interval1->first - 1 == val);
            if (left_aside && right_aside) {
                // 情况四
                int left = interval0->first, right = interval1->second;
                intervals.erase(interval0);
                intervals.erase(interval1);
                intervals.emplace(left, right);
            }
            else if (left_aside) {
                // 情况二
                ++interval0->second;
            }
            else if (right_aside) {
                // 情况三
                int right = interval1->second;
                intervals.erase(interval1);
                intervals.emplace(val, right);
            }
            else {
                // 情况五
                intervals.emplace(val, val);
            }
        }
    }
    
    vector<vector<int>> getIntervals() {
        vector<vector<int>> ans;
        for (const auto& [left, right]: intervals) {
            ans.push_back({left, right});
        }
        return ans;
    }
};

9/9 cases passed (52 ms)\
Your runtime beats 64.64 % of cpp submissions\
Your memory usage beats 54.16 % of cpp submissions (32.4 MB)

addNum() - time: O(logn)\
getIntervals() - time: O(n)

space: O(n)

# 题解2

二分查找插入 + 扫描取区间

实现直观容易，空间开销虽然也是O(n)，但却是稳定的O(n)，而不是题解1的小于O(n)。